In [1]:
#installing dependcies
!pip install PyArabic

     |████████████████████████████████| 126 kB 2.8 MB/s 


In [2]:
import requests
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import string
import sys
import pyarabic.araby as araby
import nltk 
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from textblob import Word

from sklearn import preprocessing


In [3]:
#For google colab we mount google drive 
from google.colab import drive 
drive.mount("/content/drive/")

Mounted at /content/drive/


In [4]:
tweets_data_path = "/content/drive/MyDrive/tweets_dataset.csv"

In [5]:
df=pd.read_csv(tweets_data_path, encoding='utf-8', lineterminator='\n')

In [6]:
df['tweets'] = df['tweets'].astype(str)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  458197 non-null  int64 
 1   id          458197 non-null  int64 
 2   dialect     458197 non-null  object
 3   tweets      458197 non-null  object
dtypes: int64(2), object(2)
memory usage: 14.0+ MB


In [8]:
df.loc[:,['dialect','tweets']].head(15)

,dialect,tweets
0,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,IQ,@KanaanRema مبين من كلامه خليجي
3,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺
5,IQ,@Badi9595 @KanaanRema يااخي الإرهابي اذا كان ع...
6,IQ,@SarahNadhum90 @nUBNTdfVgACYQxV مطلبي يقدم است...
7,IQ,@KanaanRema @Badi9595 خلص والله لعيونكم انا ما...
8,IQ,@SalahAlarbawi يمكن سؤال فات الكثير اللي يصور ...
9,IQ,@Eng_alow91 @cb4LwpWrS1hT5lb @EdyCohen اولا ان...


## **Text Pre-processing**

Description

**Text Cleaning**

In [9]:
###############



1. Removing Punctuations and Symbols



In [10]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [11]:
def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [12]:
df['tweets'] = df['tweets'].apply(remove_punctuations)

2. Remove Emojis



In [13]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [14]:
df['tweets'] = df['tweets'].apply(remove_emoji)

3. Stop Words Elimination




In [15]:
nltk.download('stopwords')
stop=stopwords.words('arabic')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
df['tweets'] = df['tweets'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))

4. Remove Diacritics 



In [17]:
def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', text)
    text = araby.strip_diacritics(text)
    text = araby.strip_shadda(text)
    text = araby.strip_tashkeel(text)
    return text

In [18]:
df['tweets'] = df['tweets'].apply(remove_diacritics)

5. Remove Non-Arabic Characters



In [19]:
def remove_non_arabic(text):
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = re.sub('([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+',' ',text)
    text = re.sub(r'\\u[A-Za-z0-9\\]+',' ',text)
    return text

In [20]:
df['tweets'] = df['tweets'].apply(remove_non_arabic)

5. Remove Repeated Characters 



In [21]:
def remove_repeating_characters(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub(r'(.)\1+', r'\1', text)
    return text

In [22]:
df['tweets'] = df['tweets'].apply(remove_repeating_characters)

6. Lemmatisation



In [23]:
nltk.download('wordnet')
df['tweets'] = df['tweets'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [24]:
df.loc[:,['dialect','tweets']].head(15)

,dialect,tweets
0,IQ,بالنهايه ينتفض يغير
1,IQ,يعني محسوب البشر حيونه وحشيه وتطلبون الغرب يحت...
2,IQ,مبين كلامه خليجي
3,IQ,يسلملي مرورك وروحك الحلوه
4,IQ,وين الغيبه اخ محمد
5,IQ,ياخي الارهابي اذا عراقي سعودي فلسطيني وين المش...
6,IQ,مطلبي يقدم استقالته وفوگاها اعتذار
7,IQ,خلص واله لعيونكم انا ماعندي شيء معه بالعكس متا...
8,IQ,يمكن سءال فات الكثير الي يصور شنو موقفه وكانه ...
9,IQ,اولا اني ردت رجل جنوبي والي ذكر حجابها ثانيا ا...


7. Encoding the Target Classes



In [25]:
df.dialect.unique()

array(['IQ', 'LY', 'QA', 'PL', 'SY', 'TN', 'JO', 'MA', 'SA', 'YE', 'DZ',
       'EG', 'LB', 'KW', 'OM', 'SD', 'AE', 'BH'], dtype=object)

In [30]:
#Encoding output labels
encoding = preprocessing.LabelEncoder()

#using fit transform on the data
y = encoding.fit_transform(df.dialect.values)

#replace dialec column with the encoded
df['dialect'] = y

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  458197 non-null  int64 
 1   id          458197 non-null  int64 
 2   dialect     458197 non-null  int64 
 3   tweets      458197 non-null  object
dtypes: int64(3), object(1)
memory usage: 14.0+ MB


In [32]:
df.head(15)

,Unnamed: 0,id,dialect,tweets
0,0,1175358310087892992,4,بالنهايه ينتفض يغير
1,1,1175416117793349632,4,يعني محسوب البشر حيونه وحشيه وتطلبون الغرب يحت...
2,2,1175450108898565888,4,مبين كلامه خليجي
3,3,1175471073770573824,4,يسلملي مرورك وروحك الحلوه
4,4,1175496913145217024,4,وين الغيبه اخ محمد
5,5,1175668034146643968,4,ياخي الارهابي اذا عراقي سعودي فلسطيني وين المش...
6,6,1175670153884983296,4,مطلبي يقدم استقالته وفوگاها اعتذار
7,7,1175671762580856832,4,خلص واله لعيونكم انا ماعندي شيء معه بالعكس متا...
8,8,1175715664398561280,4,يمكن سءال فات الكثير الي يصور شنو موقفه وكانه ...
9,9,1176019816072777728,4,اولا اني ردت رجل جنوبي والي ذكر حجابها ثانيا ا...


In [33]:
#save after preprocessing the tweets
df.to_csv('preprocessed_tweets.csv')  

In [34]:
#copy the csv file to google drive
!cp preprocessed_tweets.csv /content/drive/MyDrive/

In [35]:
df = pd.read_csv("preprocessed_tweets.csv")

In [36]:
df.head(15)

,Unnamed: 0,Unnamed: 0.1,id,dialect,tweets
0,0,0,1175358310087892992,4,بالنهايه ينتفض يغير
1,1,1,1175416117793349632,4,يعني محسوب البشر حيونه وحشيه وتطلبون الغرب يحت...
2,2,2,1175450108898565888,4,مبين كلامه خليجي
3,3,3,1175471073770573824,4,يسلملي مرورك وروحك الحلوه
4,4,4,1175496913145217024,4,وين الغيبه اخ محمد
5,5,5,1175668034146643968,4,ياخي الارهابي اذا عراقي سعودي فلسطيني وين المش...
6,6,6,1175670153884983296,4,مطلبي يقدم استقالته وفوگاها اعتذار
7,7,7,1175671762580856832,4,خلص واله لعيونكم انا ماعندي شيء معه بالعكس متا...
8,8,8,1175715664398561280,4,يمكن سءال فات الكثير الي يصور شنو موقفه وكانه ...
9,9,9,1176019816072777728,4,اولا اني ردت رجل جنوبي والي ذكر حجابها ثانيا ا...


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458197 entries, 0 to 458196
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    458197 non-null  int64 
 1   Unnamed: 0.1  458197 non-null  int64 
 2   id            458197 non-null  int64 
 3   dialect       458197 non-null  int64 
 4   tweets        457992 non-null  object
dtypes: int64(4), object(1)
memory usage: 17.5+ MB
